In [1]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config


import copy
import os.path as osp

import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

from mmdet.apis import set_random_seed

from mmdet.core.post_processing import bbox_nms


import base64

import glob as _glob
import os

import json
from io import BytesIO
from PIL import Image
import pandas as pd 

from ensemble_boxes import *

from mmcv.visualization import image

In [2]:
def glob(dir, pats, recursive=False):  # faster than match, python3 only
    pats = pats if isinstance(pats, (list, tuple)) else [pats]
    matches = []
    for pat in pats:
        matches += _glob.glob(os.path.join(dir, pat), recursive=recursive)
    return matches

In [3]:
## 추가수정 기존 받았던 pretrain과 매칭되는 config로 수정 
cfg = Config.fromfile('mmdetection/configs/resnest/cascade_rcnn_s101_fpn_syncbn-backbone+head_mstrain-range_1x_coco.py') 

In [4]:
@DATASETS.register_module()
class lesion_ds(CustomDataset):
    CLASSES=('01_ulcer','02_mass','04_lymph', '05_bleeding')


    def load_annotations(self, ann_file):
        
        CLASSES_dict = { '01_ulcer' : 0 , '02_mass' : 1, '04_lymph' : 2, '05_bleeding' : 3}
        
        # load image list from file
        image_list = mmcv.list_from_file(self.ann_file)
        
        data_infos = []
        
        for idx,img in enumerate(image_list):
            json_data = {}
            with open(img, "r") as json_file:
                json_data = json.load(json_file)
            
            filename = img # json에 annotation + image라서 json 자체를 filename로 주고 LoadImageFromFile을 baseline을 참조하여 custom으로 

            height = json_data['imageHeight']
            width = json_data['imageWidth']

            data_info = dict(filename=filename, width=width, height=height)

            gt_bboxes = []
            gt_labels = []

            for a_idx in range(len(json_data['shapes'])):
                gt_labels.append(CLASSES_dict[json_data['shapes'][a_idx]['label']])
                
                ## 좌표순서 좌상 우상 우하 좌하 
                ## mmdetection의 default annotation loader는 'coco_panoptic.py'에 있는데 여기 기준으로 하면 bbox에는 x1,y1,x2,y2가 담겨야함
                ## 만약에 이렇게 따로 load를 하지 않을시에는 x,y,w,h 형태로 annotaion을 생성하면 자동으로 x1,y1,x2,y2로 변환해줌 
                ori_pos = np.array(json_data['shapes'][a_idx]['points'])
                x1, y1, x2, y2 = min(ori_pos[:, 0]), min(ori_pos[:, 1]), max(ori_pos[:, 0]), max(ori_pos[:, 1])
                
                if x1 == 0 : 
                    x1 = 1 
                if y1 == 0 : 
                    y1 = 1 
                
                if x2 == width: 
                    x2 = x2 - 1 
                if y2 == width: 
                    y2 = y2-1 
            
                
                if x1==x2 or y1==y2:
                    print('Grond-truth Bounding Box 이상체크')
                    print(filename)
                     
                
                gt_bboxes.append([x1,y1,x2,y2])
                

            data_anno = dict(
                    bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                    labels=np.array(gt_labels, dtype=np.long))


            data_info.update(ann=data_anno)
            data_infos.append(data_info)
            
            if idx!=0 and idx%20000==0:
                print(str(idx)+'/'+str(len(image_list))+' load annotations END!')
            
        
        
        return data_infos

In [5]:
#sync bn하면 single gpu라 error 발생 
cfg.norm_cfg = dict(type='BN', requires_grad=True)

#class 4개로 
cfg.model.roi_head.bbox_head[0].num_classes=4

cfg.model.roi_head.bbox_head[1].num_classes=4

cfg.model.roi_head.bbox_head[2].num_classes=4

#데이터 pipeline은 config 참조 
cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

## 실제 test시에는 Test Time Augmentation 적용하는 것으로 
cfg.test_pipeline = [
    dict(type='CUSTOM_LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(600, 600),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]
cfg.data = dict(
    test=dict(pipeline=cfg.test_pipeline))


cfg.data.test.type = cfg.dataset_type


# seed set
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = [0]

print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='CascadeRCNN',
    backbone=dict(
        type='ResNeSt',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='open-mmlab://resnest101'),
        stem_channels=128,
        radix=2,
        reduction_factor=4,
        avg_down_stride=True),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=

In [6]:
# Build the detector
#model = init_detector(config, checkpoint, device='cuda:0')
checkpoint='lesion_checkpoints_ver2/epoch_10.pth'
# model = init_detector(cfg,checkpoint, device='cpu')
model = init_detector(cfg, checkpoint, device='cuda:0')
model = model.eval()
print(model.cfg)

load checkpoint from local path: lesion_checkpoints_ver2/epoch_10.pth
Config (path: mmdetection/configs/resnest/cascade_rcnn_s101_fpn_syncbn-backbone+head_mstrain-range_1x_coco.py): {'model': {'type': 'CascadeRCNN', 'backbone': {'type': 'ResNeSt', 'depth': 101, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': False, 'style': 'pytorch', 'init_cfg': {'type': 'Pretrained', 'checkpoint': 'open-mmlab://resnest101'}, 'stem_channels': 128, 'radix': 2, 'reduction_factor': 4, 'avg_down_stride': True}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0]}, 'los

In [7]:
## 추가수정 기존 받았던 pretrain과 매칭되는 config로 수정 
cfg_defor = Config.fromfile('mmdetection/configs/deformable_detr/deformable_detr_twostage_refine_r50_16x2_50e_coco.py') 

In [8]:
cfg_defor.dataset_type  = 'lesion_ds'
cfg_defor.data_root = ''

cfg_defor.work_dir = 'lesion_checkpoints_ver1'

cfg_defor.evaluation.metric = 'mAP'

#class 갯수 매칭 
cfg_defor.model.bbox_head.num_classes=4

#데이터 pipeline은 config 참조 
cfg_defor.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)


# test_pipeline, NOTE the Pad's size_divisor is different from the default
# setting (size_divisor=32). While there is little effect on the performance
# whether we use the default setting or use size_divisor=1.
cfg_defor.test_pipeline = [
    dict(type='CUSTOM_LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(736, 736),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg_defor.img_norm_cfg),
            dict(type='Pad', size_divisor=1),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

# batch size 3060에 맞게 set (기존 pretrain시에는  batch 32 사용했다고 되어있음)
cfg_defor.data = dict(
    test=dict(pipeline=cfg_defor.test_pipeline))


cfg_defor.data.test.type = cfg_defor.dataset_type

cfg_defor.model.neck.norm_cfg = dict(type='GN', num_groups=4) #batch size 매칭 

# seed set
cfg_defor.seed = 0
set_random_seed(0, deterministic=False)
cfg_defor.gpu_ids = [0]

print(f'Config:\n{cfg_defor.pretty_text}')

Config:
dataset_type = 'lesion_ds'
data_root = ''
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='AutoAugment',
        policies=[[{
            'type':
            'Resize',
            'img_scale': [(480, 1333), (512, 1333), (544, 1333), (576, 1333),
                          (608, 1333), (640, 1333), (672, 1333), (704, 1333),
                          (736, 1333), (768, 1333), (800, 1333)],
            'multiscale_mode':
            'value',
            'keep_ratio':
            True
        }],
                  [{
                      'type': 'Resize',
                      'img_scale': [(400, 4200), (500, 4200), (600, 4200)],
                      'multiscale_mode': 'value',
                      'keep_ratio': True
                  }, {
                    

In [9]:
# Build the detector
#model_defor = init_detector(cfg_defor,checkpoint, device='cpu')
checkpoint='lesion_checkpoints_ver1/epoch_50.pth'
model_defor = init_detector(cfg_defor,checkpoint, device='cuda:0')
model_defor = model_defor.eval()
print(model_defor.cfg)

/opt/conda/lib/python3.7/site-packages/mmcv/cnn/bricks/transformer.py:342: UserWarning: The arguments `feedforward_channels` in BaseTransformerLayer has been deprecated, now you should set `feedforward_channels` and other FFN related arguments to a dict named `ffn_cfgs`. 
  f'The arguments `{ori_name}` in BaseTransformerLayer '
/opt/conda/lib/python3.7/site-packages/mmcv/cnn/bricks/transformer.py:342: UserWarning: The arguments `ffn_dropout` in BaseTransformerLayer has been deprecated, now you should set `ffn_drop` and other FFN related arguments to a dict named `ffn_cfgs`. 
  f'The arguments `{ori_name}` in BaseTransformerLayer '
/opt/conda/lib/python3.7/site-packages/mmcv/cnn/bricks/transformer.py:342: UserWarning: The arguments `ffn_num_fcs` in BaseTransformerLayer has been deprecated, now you should set `num_fcs` and other FFN related arguments to a dict named `ffn_cfgs`. 
  f'The arguments `{ori_name}` in BaseTransformerLayer '
/opt/conda/lib/python3.7/site-packages/mmcv/cnn/brick

load checkpoint from local path: lesion_checkpoints_ver1/epoch_50.pth
Config (path: mmdetection/configs/deformable_detr/deformable_detr_twostage_refine_r50_16x2_50e_coco.py): {'dataset_type': 'lesion_ds', 'data_root': '', 'img_norm_cfg': {'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, 'train_pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations', 'with_bbox': True}, {'type': 'RandomFlip', 'flip_ratio': 0.5}, {'type': 'AutoAugment', 'policies': [[{'type': 'Resize', 'img_scale': [(480, 1333), (512, 1333), (544, 1333), (576, 1333), (608, 1333), (640, 1333), (672, 1333), (704, 1333), (736, 1333), (768, 1333), (800, 1333)], 'multiscale_mode': 'value', 'keep_ratio': True}], [{'type': 'Resize', 'img_scale': [(400, 4200), (500, 4200), (600, 4200)], 'multiscale_mode': 'value', 'keep_ratio': True}, {'type': 'RandomCrop', 'crop_type': 'absolute_range', 'crop_size': (384, 600), 'allow_negative_crop': True}, {'type': 'Resize', 'img_scale': [(480, 

In [10]:
test_ls=glob('lesion_DB/test','*')
test_ls.sort()

In [14]:
CLASSES_dict = { 0 : '01_ulcer' , 1 : '02_mass' , 2 : '04_lymph' , 3 : '05_bleeding' }
CLASS_NAME=['01_ulcer','02_mass','04_lymph','05_bleeding']

results_for_csv = {
    'file_name':[], 'class_id':[], 'confidence':[], 'point1_x':[], 'point1_y':[],
    'point2_x':[], 'point2_y':[], 'point3_x':[], 'point3_y':[], 'point4_x':[], 'point4_y':[]
}

def IoU(box1, box2):
    # box = (x1, y1, x2, y2)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    return iou


### weighted boxes fusion 사용 
### https://github.com/ZFTurbo/Weighted-Boxes-Fusion 참조 

for t_idx, filename in enumerate(test_ls):
    boxes_list=[]
    score_list=[]
    labels_list=[]
    weights = [2, 1] ## 첫번째 model이 이후의 모델보다 valdiation score가 높았음으로 더 높게 줌 

    
    with open(filename, "r") as json_file:
            json_data = json.load(json_file)

    img_bytes = base64.b64decode(json_data['imageData'])

    img = mmcv.imfrombytes(img_bytes, flag='color',backend='pillow')

    img = img.astype(np.float32)
    
    height, width = img.shape[:2]
    
    res1_box=[]
    res1_sco=[]
    res1_lbl=[]
    
    ## output shape x1, y1, x2, y2 , confidence score
    results_1 = inference_detector(model, img)


    for cls1,res1 in enumerate(results_1):
        if res1.shape[0]!=0:
            for r1 in res1:
                ## 좌표 값 0~1로 정규화
                r1[0] = r1[0] / width
                r1[1] = r1[1] / height
                r1[2] = r1[2] / width
                r1[3] = r1[3] / height
                res1_box.append(r1[0:4].tolist())
                res1_sco.append(float(r1[4]))
                res1_lbl.append(cls1)
            
    boxes_list.append(res1_box)
    score_list.append(res1_sco)
    labels_list.append(res1_lbl)
    
#     res2_box=[]
#     res2_sco=[]
#     res2_lbl=[]
    
#     results_2 = inference_detector(model_defor, img)

    
#     for cls2,res2 in enumerate(results_2):
#         if res2.shape[0]!=0:
#             for r2 in res2:
#                 ## 좌표 값 0~1로 정규화
#                 r2[0] = r2[0] / width
#                 r2[1] = r2[1] / height
#                 r2[2] = r2[2] / width
#                 r2[3] = r2[3] / height
#                 res2_box.append(r2[0:4].tolist())
#                 res2_sco.append(float(r2[4]))
#                 res2_lbl.append(cls2)
                
#     boxes_list.append(res2_box)
#     score_list.append(res2_sco)
#     labels_list.append(res2_lbl)
    
    ## 제출 갯수 제한이 있기때문에 skip_box_thr 값을 조정하여 적정한 output 갯수(30000개정도) 나오도록 유도 
    iou_thr = 0.55
    skip_box_thr = 0.2
    sigma = 0.1
    
    # boxes, scores, labels = weighted_boxes_fusion(boxes_list, score_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
#     print(boxes)
#     print(scores)
#     print(labels)
    
#     iou_thr = 0.4
#     skip_box_thr = 0.08
    
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, score_list, labels_list, weights=None, iou_thr=iou_thr)
    
#     print(boxes)
#     print(scores)
#     print(labels)
    
    for f_idx in range(len(boxes)):
        results_for_csv['file_name'].append(filename.split('/')[-1])
        results_for_csv['class_id'].append(labels[f_idx]+1)
        results_for_csv['confidence'].append(scores[f_idx])
        results_for_csv['point1_x'].append(boxes[f_idx][0]*576)
        results_for_csv['point1_y'].append(boxes[f_idx][1]*576)
        results_for_csv['point2_x'].append(boxes[f_idx][2]*576)
        results_for_csv['point2_y'].append(boxes[f_idx][1]*576)
        results_for_csv['point3_x'].append(boxes[f_idx][2]*576)
        results_for_csv['point3_y'].append(boxes[f_idx][3]*576)
        results_for_csv['point4_x'].append(boxes[f_idx][0]*576)
        results_for_csv['point4_y'].append(boxes[f_idx][3]*576)
#         print(results_for_csv)
    
    
#     break
    
    
    
    ## 단순 threshold 값 기준으로 하면 겹치는 box가 너무많이 생겨서 발생하는 문제 해결이 안되기 때문에 NMS로 겹쳐지는 박스의 경우 제거 해주는 것이 필요해보임 
    # for cls_idx, result in enumerate(results):
    #     if result.shape[0]!=0:
    #         out_nms = nms(result[:,0:4],result[:,4],0.4)
    #         save_target = result[out_nms]
    #         for pos_idx, pos, in enumerate(save_target):
    #             results_for_csv['file_name'].append(filename.split('/')[-1])
    #             results_for_csv['class_id'].append(cls_idx+1)
    #             results_for_csv['confidence'].append(pos[4])
    #             results_for_csv['point1_x'].append(pos[0])
    #             results_for_csv['point1_y'].append(pos[1])
    #             results_for_csv['point2_x'].append(pos[2])
    #             results_for_csv['point2_y'].append(pos[1])
    #             results_for_csv['point3_x'].append(pos[2])
    #             results_for_csv['point3_y'].append(pos[3])
    #             results_for_csv['point4_x'].append(pos[0])
    #             results_for_csv['point4_y'].append(pos[3])
    
    
    ## 적정 threshold 값 모르겠으니 일단 test output 싹다 저장해서 이미지 봐도 어느정도 score가 좋을지 잘모르겠음..
    # model.show_result(img,results,score_thr=0.0,out_file='infer_show_checkpoints10/'+filename.split('/')[-1][:-5]+'.png')
    
    
    
    
    
    ## result list로 0 1 2 3  ==> 4개 class 
    ## class id 는 1부터 시작하니까 +1 해줌 
    #  0.407    0.89
    # for cls_idx, result in enumerate(results):
    #     ## output이 0이 아니고 
    #     if result.shape[0]!=0:
    #         # print(result)
    #         thre_val = result[np.where(result[:,4]>=0.407)]
    #         if thre_val.shape[0]!=0:
    #             for pos_idx, pos, in enumerate(thre_val):
    #                 results_for_csv['file_name'].append(filename.split('/')[-1])
    #                 results_for_csv['class_id'].append(cls_idx+1)
    #                 results_for_csv['confidence'].append(pos[4])
    #                 results_for_csv['point1_x'].append(pos[0])
    #                 results_for_csv['point1_y'].append(pos[1])
    #                 results_for_csv['point2_x'].append(pos[2])
    #                 results_for_csv['point2_y'].append(pos[1])
    #                 results_for_csv['point3_x'].append(pos[2])
    #                 results_for_csv['point3_y'].append(pos[3])
    #                 results_for_csv['point4_x'].append(pos[0])
    #                 results_for_csv['point4_y'].append(pos[3])
            #confidence score가 0.5보다 클 때
            # if result[np.where(result[:,4]>0.5)].shape[0]!=0:
            #         print('11')
    if t_idx%100==0:
        print(str(t_idx) + ' end!')
     
            

0 end!
100 end!
200 end!
300 end!
400 end!
500 end!
600 end!
700 end!
800 end!
900 end!
1000 end!
1100 end!
1200 end!
1300 end!
1400 end!
1500 end!
1600 end!
1700 end!
1800 end!
1900 end!
2000 end!
2100 end!
2200 end!
2300 end!
2400 end!
2500 end!
2600 end!
2700 end!
2800 end!
2900 end!
3000 end!
3100 end!
3200 end!
3300 end!
3400 end!
3500 end!
3600 end!
3700 end!
3800 end!
3900 end!
4000 end!
4100 end!
4200 end!
4300 end!
4400 end!
4500 end!
4600 end!
4700 end!
4800 end!
4900 end!
5000 end!
5100 end!
5200 end!
5300 end!
5400 end!
5500 end!
5600 end!
5700 end!
5800 end!
5900 end!
6000 end!
6100 end!
6200 end!
6300 end!
6400 end!
6500 end!
6600 end!
6700 end!
6800 end!
6900 end!
7000 end!
7100 end!
7200 end!
7300 end!
7400 end!
7500 end!
7600 end!
7700 end!
7800 end!
7900 end!
8000 end!
8100 end!
8200 end!
8300 end!
8400 end!
8500 end!
8600 end!
8700 end!
8800 end!
8900 end!
9000 end!
9100 end!
9200 end!
9300 end!
9400 end!
9500 end!
9600 end!
9700 end!
9800 end!
9900 end!
10000 end!
1

In [15]:
submission = pd.DataFrame(results_for_csv)
submission.to_csv('HJS_VER1_WBF_TH_IOU0.55_CS_TH0.2_only_resnest.csv', index=False)